In [14]:
import requests
from pprint import pprint
import json

chain = 'polygon'
contract_name = YOUR_CONTRACT_NAME
owner_address = YOUR_WALLET_ADDRESS
metadata_updatable = True
ntfport_api = NFTPORT_API_KEY

# Create contract using NFTport

In [ ]:
url = "https://api.nftport.xyz/v0/contracts"
payload = {
            "chain": chain,
            "name": contract_name,
            "symbol": "c",
            "owner_address": owner_address,
            "metadata_updatable": metadata_updatable
            }

headers = {
    'Content-Type': "application/json",
    'Prefer': "code=200",
    'Authorization': ntfport_api
    }

contact_response = requests.request("POST", url, json=payload, headers=headers)

pprint(contact_response.json())
transaction_hash = contact_response.json()['transaction_hash']

# get contact address

In [ ]:
query_url = f"https://api.nftport.xyz/v0/contracts/{transaction_hash}"

querystring = {"chain":"polygon"}

headers = {
    'Content-Type': "application/json",
    'Authorization': ntfport_api
    }

address_response = requests.request("GET", query_url, headers=headers, params=querystring)
contract_address = address_response.json()['contract_address']
pprint(address_response.json())

In [19]:
d = contact_response.json()

d.update({'contract_address':contract_address})
# save contact address
with open(f'{contract_name}_contract.json', 'w') as f:
    json.dump(d,f)

# Upload to file and metadata IPFS

In [ ]:
import requests
import glob
from pathlib import Path
import json

upload_file_url = "https://api.nftport.xyz/v0/files"
metadata_url = "https://api.nftport.xyz/v0/metadata"
fps = glob.glob(FILE_DIR)

uri_dict = {}
for i,fp in enumerate(fps):
    filename = Path(fp).name
    file = open(fp, "rb")
    headers = {
                "Authorization": ntfport_api,
                'Prefer': "code=200, example=default",
                }

    upload_response = requests.post(upload_file_url,headers = headers,files={"file": file})
    pprint(upload_response.json())
    uploaded_url = upload_response.json()['ipfs_url']
    
    idx = str(i+1).zfill(4) # make item number 1 as 0001
    body = {
          "name": f"{contract_name} #{idx}",
          "description": NFT_DESCRIPTION,
          "file_url": uploaded_url
        }



    headers = {
        'Content-Type': "application/json",
        'Authorization': ntfport_api
        }
    metadata_response = requests.request("POST", metadata_url, json=body, headers=headers)
    metadata_uri = metadata_response.json()['metadata_uri']

    pprint(metadata_response.json())
    
    uri_dict.update({filename:{"uploaded_url":uploaded_url, "metadata_uri":metadata_uri}})
    
# save uploaded url and metadata url of all NFTs    
with open('minted.json', 'w') as fp:
    json.dump(uri_dict, fp)

# Mint NFT

In [ ]:
custome_mint_url = "https://api.nftport.xyz/v0/mints/customizable"

# mint every NFTs
for value in uri_dict.values():
    metadata_uri = value["metadata_uri"]
    
    body = {
              "chain": "polygon",
              "contract_address": contract_address,
              "metadata_uri": metadata_uri,
              "mint_to_address": owner_address
            }
    headers = {
        'Content-Type': "application/json",
        'Authorization': ntfport_api
        }

    response = requests.request("POST", custome_mint_url, json=body, headers=headers)
    pprint(response.json())